In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math


In [7]:
data_dir = "../../data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
# load data
# BasicFeatures = load_obj("pre_features")

In [8]:
filename='final_edges'
final_edges = load_obj(filename)


In [22]:
result = []
for edge in final_edges:
    row= (edge[0],edge[1])
    result.append(row)

[(4696690, 1135200, 0),
 (4802205, 4047144, 0),
 (1951683, 4024907, 0),
 (3542775, 1239673, 0),
 (2778303, 1276807, 0),
 (4842507, 4546609, 0),
 (2050710, 2353972, 0),
 (3698046, 3643090, 0),
 (3672389, 3959954, 0),
 (2384341, 2356939, 0),
 (3571391, 1144219, 0),
 (451392, 203622, 0),
 (3676921, 2717113, 0),
 (2633571, 3654915, 0),
 (1999674, 1368123, 0),
 (1305133, 2277481, 0),
 (2357487, 2574465, 0),
 (1970103, 1831611, 0),
 (2524292, 2878454, 0),
 (2978385, 4050596, 0),
 (374254, 1852774, 0),
 (4450015, 902139, 0),
 (3062559, 1504438, 0),
 (1904424, 1655118, 0),
 (3171050, 2470187, 0),
 (3988798, 4375563, 0),
 (2292299, 1743138, 0),
 (399205, 4095299, 0),
 (3726633, 4723423, 0),
 (222174, 1454341, 0),
 (4648776, 3454963, 0),
 (506000, 68331, 0),
 (4724429, 1533188, 0),
 (1738774, 2604202, 0),
 (3408288, 710101, 0),
 (2193015, 3960751, 0),
 (2258741, 1183256, 0),
 (203801, 497600, 0),
 (1776535, 4478960, 0),
 (481709, 3597707, 0),
 (2699444, 4455643, 0),
 (1129658, 1700262, 0),
 (176

In [25]:
import pandas as pd
def get_common_neighbours(node1, node2):
    try:
        n1 = pre_features[node1]
        n2 = pre_features[node2]
        common_neighors = list(set(n1[2]).intersection(n2[2]))
        return common_neighors
    except:
        return []

training_df = pd.DataFrame()

# prepare dataset for ANN
for edge in final_edges:  
#     print(source, sink, label)
    source = edge[0]
    sink = edge[1]
    label =edge[2]
    common_neighbours = get_common_neighbours(source, sink)
    if len(common_neighbours)>0:
        print('hello')
        print(len(common_neighbours))
    if len(common_neighbours)>=100:
        common_neighbours = common_neighbours[:100]
        
    else:
        pass
#     print(common_neighbours)
    common_neighbours.append(label)
    row_df = pd.DataFrame(common_neighbours)
    training_df = training_df.append(row_df)
    

In [19]:
filename='edges_10w'
with open(filename + ".txt", 'w', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    f_csv.writerows()

In [20]:
training_df_ANN.head

<bound method NDFrame.head of     0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
.. ..
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1

[100000 rows x 1 columns]>

In [16]:
test=[1,2,3,4]
test[:2]
test.append

TypeError: can only concatenate list (not "int") to list

In [ ]:
# 算二级的common neighbour的个数
def 

In [15]:
import pandas as pd
# in-degree statistics
df = pd.DataFrame(BasicFeatures)
df =  df.T
df.rename(columns={0: 'num_of_neighbours',
                   1: 'log_num_of_neighbours',
                  2: 'list_of_neighbours',
                  3: 'list_of_in_neighbours',
                  4: 'num_of_in_neighbours',
                  5: 'list_of_out_neighbours',
                  6: 'num_of_out_neighbours'}, inplace=True)

In [45]:
df.loc[[2]]

,num_of_neighbours,log_num_of_neighbours,list_of_neighbours,list_of_in_neighbours,num_of_in_neighbours,list_of_out_neighbours,num_of_out_neighbours
2,2,0.910239,"[20388, 3360411]","[3360411, 20388]",2,[],0


In [67]:
new_df = df.loc[[2]]
new = new_df.append(df.loc[[1]])

In [69]:
new

,num_of_neighbours,log_num_of_neighbours,list_of_neighbours,list_of_in_neighbours,num_of_in_neighbours,list_of_out_neighbours,num_of_out_neighbours
2,2,0.910239,"[20388, 3360411]","[3360411, 20388]",2,[],0
1,3,0.721348,"[1247754, 2382107, 4588320]","[4588320, 1247754, 2382107]",3,[],0


In [72]:
new_df = pd.DataFrame()
# 找到所有的training data中有test data中有edge的边
for node_in_test_data in nodes_in_test_data:
    node_in_test_data = int(node_in_test_data)

    try:
        df_row = df.loc[[node_in_test_data]]
#         print(df_row)
        new_df = new_df.append(df_row)
#         print(new_df)
    except:
        continue

required_set = set()
for index, row in new_df.iterrows():
#     print(index)
#     print(row['list_of_neighbours'])
    required_set.add(index)
    for node in row['list_of_neighbours']:
        required_set.add(node)
len(required_set)
# 利用已经有的边生成新的训练集

data_dir = "../../data/"

with open(data_dir + "edges.txt", "r") as f:
    read_rows = f.readlines()


1222124

In [116]:
len(nodes_in_test_data)
nodes_in_test_data.astype(int8)

AttributeError: 'set' object has no attribute 'astype'

In [ ]:
rows = [i.split() for i in read_rows[0:]]
edges_of_all_test_nodes_related=list()

for row in rows:
    row[0] = int(row[0])
    row[1] = int(row[1])
    edge = (row[0], row[1])
    if str(row[0]) in nodes_in_test_data or str(row[1]) in nodes_in_test_data:
        edges_of_all_test_nodes_related.append(edge)

In [110]:
len(rows)

24004361

In [111]:
len(edges_of_all_test_nodes_related)

0

In [ ]:
# 生成negative sample
for 

In [5]:
import csv
import time
'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
#     headers = ['id','Prediction']
    with open(filename + str(nowtime()) + ".txt", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
#         f_csv.writerow(headers)
        for row in result:
            f_csv.writerow(result)

save_prediction_to_csv(Edges,'edges')

TypeError: writerows() takes no keyword arguments